# Lab 1: Building Reactive Agents

In this lab, you'll build reactive agents from scratch and understand the stimulus-response pattern.

## Learning Objectives
- Understand reactive agent architecture
- Implement rule-based decision making
- Build multiple agent types (thermostat, trading bot, game AI)
- Test agents in simulated environments

## Part 1: Basic Reactive Agent

A reactive agent maps percepts directly to actions using rules.

In [ ]:
from abc import ABC, abstractmethod
from typing import Any, Dict

class ReactiveAgent(ABC):
    """Base class for reactive agents"""
    
    def __init__(self, name: str):
        self.name = name
        self.action_history = []
    
    @abstractmethod
    def perceive(self, environment: Dict) -> Dict:
        """Get percepts from environment"""
        pass
    
    @abstractmethod
    def decide(self, percepts: Dict) -> str:
        """Choose action based on percepts"""
        pass
    
    def act(self, action: str):
        """Execute action and record it"""
        self.action_history.append(action)
        return action
    
    def run_cycle(self, environment: Dict) -> str:
        """One perception-action cycle"""
        percepts = self.perceive(environment)
        action = self.decide(percepts)
        return self.act(action)

## Part 2: Thermostat Agent

A simple agent that controls temperature.

In [ ]:
class ThermostatAgent(ReactiveAgent):
    """A reactive thermostat agent"""
    
    def __init__(self, name: str, target_temp: float = 21.0, tolerance: float = 1.0):
        super().__init__(name)
        self.target_temp = target_temp
        self.tolerance = tolerance
    
    def perceive(self, environment: Dict) -> Dict:
        return {
            'current_temp': environment.get('temperature', 20),
            'humidity': environment.get('humidity', 50)
        }
    
    def decide(self, percepts: Dict) -> str:
        temp = percepts['current_temp']
        
        if temp < self.target_temp - self.tolerance:
            return 'HEAT_ON'
        elif temp > self.target_temp + self.tolerance:
            return 'COOL_ON'
        else:
            return 'MAINTAIN'

# Test the thermostat
thermostat = ThermostatAgent('Living Room', target_temp=21)

test_environments = [
    {'temperature': 18, 'humidity': 45},
    {'temperature': 21, 'humidity': 50},
    {'temperature': 25, 'humidity': 60},
]

for env in test_environments:
    action = thermostat.run_cycle(env)
    print(f"Temp: {env['temperature']}°C → Action: {action}")

## Exercise 1: Trading Bot Agent

Implement a simple trading agent that:
- Buys when price drops below a threshold
- Sells when price rises above a threshold
- Holds otherwise

In [ ]:
class TradingAgent(ReactiveAgent):
    """A simple reactive trading agent"""
    
    def __init__(self, name: str, buy_threshold: float, sell_threshold: float):
        super().__init__(name)
        self.buy_threshold = buy_threshold
        self.sell_threshold = sell_threshold
        self.position = 0  # Number of shares held
    
    def perceive(self, environment: Dict) -> Dict:
        # YOUR CODE HERE
        # Return dict with 'price' and 'volume'
        pass
    
    def decide(self, percepts: Dict) -> str:
        # YOUR CODE HERE
        # Return 'BUY', 'SELL', or 'HOLD'
        pass

# Test your trading agent
trader = TradingAgent('SimpleTrder', buy_threshold=95, sell_threshold=105)

market_data = [
    {'price': 100, 'volume': 1000},
    {'price': 90, 'volume': 1500},
    {'price': 110, 'volume': 2000},
    {'price': 98, 'volume': 800},
]

for data in market_data:
    action = trader.run_cycle(data)
    print(f"Price: ${data['price']} → Action: {action}")

## Part 3: Rule-Based System

A more flexible approach using explicit rules.

In [ ]:
from dataclasses import dataclass
from typing import Callable, List

@dataclass
class Rule:
    """A condition-action rule"""
    name: str
    condition: Callable[[Dict], bool]
    action: str
    priority: int = 0

class RuleBasedAgent(ReactiveAgent):
    """Agent that uses explicit rules"""
    
    def __init__(self, name: str, rules: List[Rule]):
        super().__init__(name)
        # Sort rules by priority (highest first)
        self.rules = sorted(rules, key=lambda r: r.priority, reverse=True)
    
    def perceive(self, environment: Dict) -> Dict:
        return environment
    
    def decide(self, percepts: Dict) -> str:
        for rule in self.rules:
            if rule.condition(percepts):
                print(f"  Triggered rule: {rule.name}")
                return rule.action
        return 'NO_ACTION'

# Example: Home Security Agent
security_rules = [
    Rule('fire_alarm', lambda p: p.get('smoke', False), 'CALL_FIRE_DEPT', priority=100),
    Rule('intrusion', lambda p: p.get('motion', False) and not p.get('owner_home', True), 'ALERT_OWNER', priority=50),
    Rule('night_lights', lambda p: p.get('dark', False) and p.get('owner_home', False), 'LIGHTS_ON', priority=10),
]

security = RuleBasedAgent('HomeSecurity', security_rules)

scenarios = [
    {'smoke': True, 'motion': False, 'owner_home': True, 'dark': False},
    {'smoke': False, 'motion': True, 'owner_home': False, 'dark': True},
    {'smoke': False, 'motion': False, 'owner_home': False, 'dark': True},
]

for i, scenario in enumerate(scenarios):
    print(f"\nScenario {i+1}: {scenario}")
    action = security.run_cycle(scenario)
    print(f"Action: {action}")

## Exercise 2: Game AI Agent

Create a reactive agent for a simple game where:
- If enemy is close and health is low → FLEE
- If enemy is close and health is high → ATTACK
- If health is low and no enemy → HEAL
- If treasure nearby → COLLECT
- Otherwise → EXPLORE

In [ ]:
# YOUR CODE HERE
# Create game_rules list with Rule objects
# Create RuleBasedAgent with those rules
# Test with various game states

game_rules = [
    # YOUR CODE HERE
]

game_ai = RuleBasedAgent('GameAI', game_rules)

# Test scenarios
game_states = [
    {'enemy_distance': 5, 'health': 20, 'treasure_nearby': False},
    {'enemy_distance': 5, 'health': 80, 'treasure_nearby': False},
    {'enemy_distance': 100, 'health': 30, 'treasure_nearby': False},
    {'enemy_distance': 100, 'health': 80, 'treasure_nearby': True},
]

## Part 4: Environment Simulation

Let's create a simple environment that agents can interact with.

In [ ]:
import random

class SimpleEnvironment:
    """A simple simulated environment"""
    
    def __init__(self):
        self.temperature = 20.0
        self.time_step = 0
    
    def get_state(self) -> Dict:
        return {
            'temperature': self.temperature,
            'time': self.time_step
        }
    
    def apply_action(self, action: str):
        if action == 'HEAT_ON':
            self.temperature += 0.5
        elif action == 'COOL_ON':
            self.temperature -= 0.5
        
        # Natural drift
        self.temperature += random.uniform(-0.2, 0.2)
        self.time_step += 1
    
    def run_simulation(self, agent: ReactiveAgent, steps: int = 20):
        history = []
        
        for _ in range(steps):
            state = self.get_state()
            action = agent.run_cycle(state)
            self.apply_action(action)
            history.append({
                'time': self.time_step,
                'temp': state['temperature'],
                'action': action
            })
        
        return history

# Run simulation
env = SimpleEnvironment()
env.temperature = 15.0  # Start cold

thermostat = ThermostatAgent('SmartThermo', target_temp=21)
history = env.run_simulation(thermostat, steps=30)

# Visualize
import matplotlib.pyplot as plt

times = [h['time'] for h in history]
temps = [h['temp'] for h in history]

plt.figure(figsize=(10, 4))
plt.plot(times, temps, 'b-', label='Temperature')
plt.axhline(y=21, color='r', linestyle='--', label='Target')
plt.xlabel('Time Step')
plt.ylabel('Temperature (°C)')
plt.title('Thermostat Agent Simulation')
plt.legend()
plt.grid(True)
plt.show()

## Challenge Exercise

Create a multi-agent simulation with:
1. Multiple thermostat agents in different rooms
2. Heat transfer between rooms
3. External temperature influence

Visualize how the agents coordinate (or don't) to maintain comfort.

In [ ]:
# YOUR CODE HERE - Challenge Exercise